In [1]:
import opensfm.reconstruction as rec
from opensfm import dataset
from opensfm import pysfm
from opensfm import tracking

import logging
from timeit import default_timer as timer
import numpy as np
import math
import time

logger = logging.getLogger(__name__)

In [25]:
def incremental_reconstruction(data, tracks_manager, foc_len):
    """Run the entire incremental reconstruction pipeline."""
    logger.info("Starting incremental reconstruction")
    report = {}
    chrono = rec.Chronometer()

    images = tracks_manager.get_shot_ids()
    
    if not data.reference_lla_exists():
        data.invent_reference_lla(images)

    remaining_images = set(images)
    camera_priors = data.load_camera_models()
    for i in camera_priors.keys():
        camera_priors[i].focal = foc_len
    gcp = data.load_ground_control_points()
    common_tracks = tracking.all_common_tracks(tracks_manager)
    reconstructions = []
    pairs = rec.compute_image_pairs(common_tracks, camera_priors, data)
    chrono.lap('compute_image_pairs')
    report['num_candidate_image_pairs'] = len(pairs)
    report['reconstructions'] = []
    
    t0 = time.time()
    
    for im1, im2 in pairs:
        if im1 in remaining_images and im2 in remaining_images:
            "Running boostrap and BA"
            rec_report = {}
            report['reconstructions'].append(rec_report)
            _, p1, p2 = common_tracks[im1, im2]
            reconstruction, graph_inliers, rec_report['bootstrap'] = rec.bootstrap_reconstruction(
                data, tracks_manager, camera_priors, im1, im2, p1, p2)

            if reconstruction:
                remaining_images.remove(im1)
                remaining_images.remove(im2)
                reconstruction, rec_report['grow'] = rec.grow_reconstruction(
                    data, tracks_manager, graph_inliers, reconstruction, remaining_images, camera_priors, gcp)
                reconstructions.append(reconstruction)
                reconstructions = sorted(reconstructions,
                                         key=lambda x: -len(x.shots))
                rec_report['stats'] = rec.compute_statistics(reconstruction, graph_inliers)
                logger.info(rec_report['stats'])
                
    t1 = time.time()
    
    print("time: ", t1-t0)

    for k, r in enumerate(reconstructions):
        logger.info("Reconstruction {}: {} images, {} points".format(
            k, len(r.shots), len(r.points)))
    logger.info("{} partial reconstructions in total.".format(
        len(reconstructions)))
    chrono.lap('compute_reconstructions')
    report['wall_times'] = dict(chrono.lap_times())
    report['not_reconstructed_images'] = list(remaining_images)
    return report, reconstructions

In [81]:
data = dataset.DataSet('data/iphone11pro_wide_office')
tracks_manager = data.load_tracks_manager()

# tracks_manager = pysfm.TracksManager.instanciate_from_file('/Users/jarrodhaas/torus-fast-track-dependencies/OpenSfM/data/gopro8_grass/tracks.csv')

In [ ]:
report, reconstructions = incremental_reconstruction(data, tracks_manager, foc_len=0.7)

2020-07-31 16:47:59,762 INFO: Starting incremental reconstruction
2020-07-31 16:48:04,060 INFO: Starting reconstruction with 0152.png and 0176.png
2020-07-31 16:48:04,087 INFO: Two-view reconstruction inliers: 1251 / 1265
2020-07-31 16:48:04,229 INFO: Triangulated: 1245
2020-07-31 16:48:04,248 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 3.561502e+01, Final cost: 3.084831e+01, Termination: CONVERGENCE
2020-07-31 16:48:04,465 DEBUG: Ceres Solver Report: Iterations: 3, Initial cost: 3.182228e+01, Final cost: 3.000421e+01, Termination: CONVERGENCE
2020-07-31 16:48:04,541 DEBUG: Ceres Solver Report: Iterations: 11, Initial cost: 6.572109e+01, Final cost: 5.624250e+01, Termination: CONVERGENCE
2020-07-31 16:48:04,546 INFO: Removed outliers: 1
2020-07-31 16:48:04,552 INFO: -------------------------------------------------------
2020-07-31 16:48:04,570 INFO: 0175.png resection inliers: 1108 / 1118
2020-07-31 16:48:04,595 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 4.5

2020-07-31 16:48:14,406 INFO: Removed outliers: 1
2020-07-31 16:48:14,413 INFO: -------------------------------------------------------
2020-07-31 16:48:14,448 INFO: 0164.png resection inliers: 1794 / 1830
2020-07-31 16:48:14,487 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 1.499805e+02, Final cost: 1.303304e+02, Termination: CONVERGENCE
2020-07-31 16:48:14,488 INFO: Adding 0164.png to the reconstruction
2020-07-31 16:48:14,583 DEBUG: Local bundle sets: interior 14  boundary 0  other 0
2020-07-31 16:48:15,222 DEBUG: Ceres Solver Report: Iterations: 11, Initial cost: 1.525303e+03, Final cost: 1.488610e+03, Termination: NO_CONVERGENCE
2020-07-31 16:48:15,270 INFO: Removed outliers: 2
2020-07-31 16:48:15,276 INFO: -------------------------------------------------------
2020-07-31 16:48:15,304 INFO: 0163.png resection inliers: 2016 / 2050
2020-07-31 16:48:15,350 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 1.484484e+02, Final cost: 1.328896e+02, Termination: CONVERG

2020-07-31 16:48:41,131 INFO: 0153.png resection inliers: 2135 / 2180
2020-07-31 16:48:41,189 DEBUG: Ceres Solver Report: Iterations: 5, Initial cost: 2.215703e+02, Final cost: 1.922331e+02, Termination: CONVERGENCE
2020-07-31 16:48:41,191 INFO: Adding 0153.png to the reconstruction
2020-07-31 16:48:41,278 DEBUG: Local bundle sets: interior 26  boundary 0  other 0
2020-07-31 16:48:42,547 DEBUG: Ceres Solver Report: Iterations: 7, Initial cost: 4.115372e+03, Final cost: 4.075901e+03, Termination: CONVERGENCE
2020-07-31 16:48:42,643 INFO: Removed outliers: 0
2020-07-31 16:48:42,646 INFO: -------------------------------------------------------
2020-07-31 16:48:42,678 INFO: 0169.png resection inliers: 2107 / 2151
2020-07-31 16:48:42,729 DEBUG: Ceres Solver Report: Iterations: 4, Initial cost: 1.552785e+02, Final cost: 1.479803e+02, Termination: CONVERGENCE
2020-07-31 16:48:42,731 INFO: Adding 0169.png to the reconstruction
2020-07-31 16:48:42,827 DEBUG: Local bundle sets: interior 27  boun

In [ ]:
from opensfm import io
# with open(data.profile_log(), 'a') as fout:
#             fout.write('reconstruct: {0}\n'.format(end - start))
data.save_reconstruction(reconstructions)
data.save_report(io.json_dumps(report), 'reconstruction.json')

In [110]:
(1920-1153) / 1920

0.39947916666666666

In [77]:
0.7 * 1920

1344.0

# How are we calculating bearings?

In [16]:
camera_priors = data.load_camera_models()

In [152]:
common_tracks = tracking.all_common_tracks(tracks_manager)
_, p1, p2 = common_tracks['0004.png', '0030.png']

In [27]:
camera_priors['v2 unknown unknown 1920 1080 perspective 0'].get_K()

array([[0.8, 0. , 0. ],
       [0. , 0.8, 0. ],
       [0. , 0. , 1. ]])

In [158]:
K = camera_priors['v2 unknown unknown 1920 1080 perspective 0'].get_K_in_pixel_coordinates(1920,1080)

In [159]:
K

array([[1.632e+03, 0.000e+00, 9.600e+02],
       [0.000e+00, 1.632e+03, 5.400e+02],
       [0.000e+00, 0.000e+00, 1.000e+00]])

In [90]:
def sqrNorm(a):
    return a[0]**2 + a[1]**2

In [83]:
b1 = a[0] / (0.85) * 0.98956846

In [88]:
0.04465131 / 0.98956846 * 0.85

0.038353701673151545

In [148]:
point = np.array([ p1[0,0] , p1[0,1]])

inv_norm = 1.0 / ((sqrNorm(point) + 1.0)**0.5)

In [149]:
a2[0] * inv_norm

0.038258862027737504

In [150]:
inv_norm

0.9924304410101045

In [162]:
from opensfm import exif

In [ ]:
d = exif.extract_exif_from_file('data/office/images/0006.png')

In [250]:
0.5150781691074733* 1920

988.9500846863486

# Camera Metadata Test

In [2]:
import copy
import logging
import time

from opensfm import dataset
from opensfm import exif

In [3]:
logger = logging.getLogger(__name__)
logging.getLogger("exifread").setLevel(logging.WARNING)

In [4]:
def _extract_exif(image, data):
     # EXIF data in Image
    d = exif.extract_exif_from_file(data.open_image_file(image))

    # Image Height and Image Width
    if d['width'] <= 0 or not data.config['use_exif_size']:
        d['height'], d['width'] = data.image_size(image)

    d['camera'] = exif.camera_id(d)

    return d


start = time.time()
data = dataset.DataSet('data/office')

exif_overrides = {}
if data.exif_overrides_exists():
    exif_overrides = data.load_exif_overrides()

camera_models = {}
for image in data.images():
#     if data.exif_exists(image):
#         logging.info('Loading existing EXIF for {}'.format(image))
#         d = data.load_exif(image)
#     else:
        
    logging.info('Extracting EXIF for {}'.format(image))
    d = _extract_exif(image, data)

    if image in exif_overrides:
        
        d.update(exif_overrides[image])

    data.save_exif(image, d)

    if d['camera'] not in camera_models:
        
        camera = exif.camera_from_exif_metadata(d, data)
        camera_models[d['camera']] = camera
        break

# Override any camera specified in the camera models overrides file.
if data.camera_models_overrides_exists():
    overrides = data.load_camera_models_overrides()
    if "all" in overrides:
        for key in camera_models:
            camera_models[key] = copy.copy(overrides["all"])
            camera_models[key].id = key
    else:
        for key, value in overrides.items():
            camera_models[key] = value
data.save_camera_models(camera_models)

end = time.time()
with open(data.profile_log(), 'a') as fout:
    fout.write('extract_metadata: {0}\n'.format(end - start))



In [5]:
data.config

{'use_exif_size': True,
 'default_focal_prior': 0.9,
 'feature_type': 'HAHOG',
 'feature_root': 1,
 'feature_min_frames': 4000,
 'feature_process_size': 2048,
 'feature_use_adaptive_suppression': False,
 'sift_peak_threshold': 0.1,
 'sift_edge_threshold': 10,
 'surf_hessian_threshold': 3000,
 'surf_n_octaves': 4,
 'surf_n_octavelayers': 2,
 'surf_upright': 0,
 'akaze_omax': 4,
 'akaze_dthreshold': 0.001,
 'akaze_descriptor': 'MSURF',
 'akaze_descriptor_size': 0,
 'akaze_descriptor_channels': 3,
 'akaze_kcontrast_percentile': 0.7,
 'akaze_use_isotropic_diffusion': False,
 'hahog_peak_threshold': 1e-05,
 'hahog_edge_threshold': 10,
 'hahog_normalize_to_uchar': True,
 'lowes_ratio': 0.8,
 'matcher_type': 'FLANN',
 'symmetric_matching': True,
 'flann_algorithm': 'KMEANS',
 'flann_branching': 8,
 'flann_iterations': 10,
 'flann_tree': 8,
 'flann_checks': 20,
 'bow_file': 'bow_hahog_root_uchar_10000.npz',
 'bow_words_to_match': 50,
 'bow_num_checks': 20,
 'bow_matcher_type': 'FLANN',
 'vlad_

# focal length

In [129]:
(1080 / 2) / (math.tan(math.pi*43.60281897/360.0))

1350.0000000923687

In [122]:
2*(30*math.tan(0.5*61.92751306/57.296))

35.99983029809857

In [267]:
37.21 * 1920 / 36

1984.5333333333333

In [ ]:
f_px = f__mm * image_width / sensor_width

In [240]:
61 * 16/9

108.44444444444444

In [239]:
3.99*(8.47)

33.795300000000005

In [238]:
# f_mm to f_px
3.99*(8.47)/36 * 1920

1802.4160000000002

In [237]:
# ratio of sensor widths
36/4.25

8.470588235294118